In [1]:
import os
import sys
import json
import threading
from datetime import datetime

class key_value_datastore:
    
    def __init__(self,dir_path=os.getcwd(),filename="file.json"):
        self.lock=threading.Lock()
        self.dir_path=dir_path
        self.filename=filename
        if os.path.exists(self.dir_path):
            if os.path.exists(os.path.join(self.dir_path,self.filename)):
                pass
            else:
                try:
                    with open(os.path.join(self.dir_path,self.filename),"w") as f:
                        self.json_object={}
                        json.dump(self.json_object,fp=f,indent=3)
                except:
                    raise RuntimeError("Error in opening the file")
        else:
            os.mkdir(self.dir_path)
            try:
                with  open(os.path.join(self.dir_path,self.filename),"w") as f:
                    self.json_object={}
                    json.dump(self.json_object,fp=f,indent=3)
            except:
                raise RuntimeError("Error in opening the file")
            
            
    def create(self,key,value,ttl=0):
        self.lock.acquire()
        try:
            if os.path.getsize(os.path.join(self.dir_path,self.filename))>10**9:
                raise RuntimeError("The datastore has no space")
                
            with open(os.path.join(self.dir_path,self.filename),"r") as f:
                    json_object=json.load(f)
                    
            if key not in json_object.keys():
                if type(key)==str:
                    if len(key)<32:
                        self.created_time=datetime.now()
                        json_object[key]=[value,ttl,str(self.created_time)]
                        if sys.getsizeof(json_object[key])>16000:
                            raise RuntimeError("value size is more than 16KB")
                    else:
                        raise ValueError("key should contain only 32 characters")
                else:
                    raise TypeError("key should be of string type")
            else:
                raise RuntimeError("key already exists")
            with open(os.path.join(self.dir_path,self.filename),"w") as f:
                json.dump(json_object,fp=f,indent=3)
        finally:
            self.lock.release()
            
            
    def __getattr__(self,key):
        self.lock.acquire()
        try:
            with open(os.path.join(self.dir_path,self.filename),"r") as f:
                    json_object=json.load(f)
                    
            if key in json_object.keys():
        #Logic used for setting time to live property
                if json_object[key][1] != 0:
                    curr_time=datetime.now()
                    time_diff=curr_time - datetime.strptime(json_object[key][2],"%Y-%m-%d %H:%M:%S.%f")
                    if time_diff.seconds < json_object[key][1]:
                        return json_object[key][0]
                    else:
                        raise RuntimeError("Key got expired")
                else:
                    return json_object[key][0]
            else:
                raise KeyError("key not found")
        finally:
            self.lock.release()
    
    
    def __delitem__(self,key):
        self.lock.acquire()
        try:
            with open(os.path.join(self.dir_path,self.filename),"r") as f:
                    json_object=json.load(f)
                    
            if key in json_object.keys():
                if json_object[key][1]== 0:
                    json_object.pop(key)
                    with open(os.path.join(self.dir_path,self.filename),"w") as f:
                        json.dump(json_object,fp=f,indent=3)
                else:
            #Logic used for setting time to live property
                    self.del_curr_time=datetime.now()
                    self.diff_time=self.del_curr_time - datetime.strptime(json_object[key][2],"%Y-%m-%d %H:%M:%S.%f")
                    if self.diff_time.seconds < json_object[key][1]:
                        json_object.pop(key)
                        with open(os.path.join(self.dir_path,self.filename),"w") as f:
                            json.dump(json_object,fp=f,indent=3)
                    else:
                        raise RuntimeError("deletion failed!....key got expired")
            else:
                raise KeyError("key not found")
        finally:
            self.lock.release()
    

Testing the data store functional requirements

In [2]:
#1.   This data store has exposed as a library to clients that can be instantiated by using  a class "key_value_datastore" and
#can work with the data store.

#2.   It can be initialized using an optional filepath or it will create itself in a reasonable location on the laptop

student_data_store = key_value_datastore()

#student_data_store = key_value_datastore("C:\data store","file.json")

In [3]:
#3.   A new key-value pair can be added to the data store using the Create operation.

student_data_store.create("Name","Sasidharan M")
student_data_store.create("Roll_No","2017502560")
student_data_store.create("College","MIT")

In [4]:
#4.  Shows error when the key is already existed

student_data_store.create("Name","sasi")

RuntimeError: key already exists

In [5]:
#5.  Shows error when the key is not string and has more than 32 chars.

#6.  Shows error when the value size is more than 16kb

student_data_store.create(1,"int")

TypeError: key should be of string type

In [6]:
#7.  A read operation on a key can be performed by providing a key

student_data_store.Name

'Sasidharan M'

In [7]:
#8.  A Delete operation can be performed by providing the key

del student_data_store["College"]

In [8]:
#9.  Time to live property can be set to key by providing the number of seconds the key need to be retaind in the data store 
#as an argument to the function.

import time
student_data_store.create("Section","A",ttl=15)
time.sleep(10)
student_data_store.Section

'A'

In [9]:
student_data_store.create("Place","Chennai",5)
time.sleep(6)
student_data_store.Place

RuntimeError: Key got expired

In [10]:
#10.  A client process is allowed to access the data store using multiple threads.

import threading
t1=threading.Thread(target=student_data_store.create("State","Tamil Nadu"))
t2=threading.Thread(target=(student_data_store.__delitem__("State")))
t1.start()
t2.start()
t1.join()
t2.join()